### Create a SQLite database from CSVs with Python
The program is for building a database in SQLite by importing csv files downloaded from Bank of America (BoA) with python. 

After moving into a new place, I want to set up an account book for self-review. I wrote a program to keep my books organized and up-to-date. All we need to do is to download the csv file of the latest statement to the "boa" folder in the same directory and run the program step by step. I hope it is helpful.



Some reference:
* https://mungingdata.com/sqlite/create-database-load-csv-python/, 
* https://www.projectpro.io/recipes/read-csv-data-from-local-system-and-replace-and-rename-columns

In [1]:
# import the modules
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import OperationalError
import os

months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

In [2]:
# Import cvs files
year = input("Please enter the year: ")
month = int(input("Please enter the month (1-12): "))
card_4digits = input("Please enter the last four digits of your credit card: ")

try:
    files_dir = os.path.join("boa")
    filename = f"{months[month - 1]}{year}_{card_4digits}.csv"
    df = pd.read_csv(os.path.join(files_dir, filename))
except FileNotFoundError:
    print("No such file or directory")
except IndexError:
    print("Not valid month")
except:
    print("Unknown error")

df.head()

Please enter the year:  2025
Please enter the month (1-12):  4
Please enter the last four digits of your credit card:  1234


,Posted Date,Reference Number,Payee,Address,Amount
0,04/21/2025,12345678912345678912344,PAYMENT - THANK YOU,NaN,314.81
1,04/16/2025,12345678912345678912345,APPLE.COM/BILL 866-712-7753 CA,866-712-7753 CA,-9.99
2,04/12/2025,12345678912345678912346,TARGET 00012385 IRVINE CA,IRVINE CA,-43.58
3,04/12/2025,12345678912345678912347,HOMEGOODS # 0728 IRVINE CA,IRVINE CA,-64.64
4,04/09/2025,12345678912345678912348,TRADER JOE'S #210 QPS IRVINE CA,IRVINE CA,-38.23


In [3]:
# Rename the column: remove the space and name it simplly
df = df.rename(columns = {"Posted Date": "date", "Reference Number": "number", "Payee": "payee", "Address": "address", "Amount": "amount"})

# Change the format of date
df['date'] = pd.to_datetime(df['date'])
df.head()

,date,number,payee,address,amount
0,2025-04-21,12345678912345678912344,PAYMENT - THANK YOU,NaN,314.81
1,2025-04-16,12345678912345678912345,APPLE.COM/BILL 866-712-7753 CA,866-712-7753 CA,-9.99
2,2025-04-12,12345678912345678912346,TARGET 00012385 IRVINE CA,IRVINE CA,-43.58
3,2025-04-12,12345678912345678912347,HOMEGOODS # 0728 IRVINE CA,IRVINE CA,-64.64
4,2025-04-09,12345678912345678912348,TRADER JOE'S #210 QPS IRVINE CA,IRVINE CA,-38.23


In [4]:
# Check for missing data
df.isnull().sum()

date       0
number     0
payee      0
address    1
amount     0
dtype: int64

In [5]:
# Fill the missing data
df.address = df.address.fillna("NO RECORD")
df.isnull().sum()

date       0
number     0
payee      0
address    0
amount     0
dtype: int64

In [6]:
# Connect or create to the database
name_of_database = input("Please enter the name of the database: ")
connection = sqlite3.connect(f"{name_of_database}.db")

Please enter the name of the database:  demo


In [7]:
# Create a new table or if the table already exists, enter the name of the table
name_of_new_table = input("Please enter the name of the new table in the database: ")

try:
    # Table Definition
    new_table = f'''CREATE TABLE {name_of_new_table} (date TEXT, 
                                                      number TEXT, 
                                                      payee TEXT, 
                                                      address TEXT, 
                                                      amount REAL)'''
    connection.execute(new_table)

except OperationalError:  # import from sqlite3 modules
    print("table already exists")
    
except: 
    print("Unknow error")

Please enter the name of the new table in the database:  demo_table


In [8]:
df.to_sql(f"{name_of_new_table}", connection, if_exists = "append", index = False)
connection.close()

### Check the update of database
The above code completed building the database. The following is to make sure the data successfully update.

In [9]:
connection = sqlite3.connect(f"{name_of_database}.db")
df2 = pd.read_sql(f'''SELECT * FROM {name_of_new_table}''', connection)
connection.close()
df2.tail()

,date,number,payee,address,amount
4,2025-04-09 00:00:00,12345678912345678912348,TRADER JOE'S #210 QPS IRVINE CA,IRVINE CA,-38.23
5,2025-04-09 00:00:00,12345678912345678912349,Uniqlo USA LLC New York NY,New York NY,-118.31
6,2025-04-09 00:00:00,12345678912345678912350,Nintendo CD782790275 800-2553700 WA,800-2553700 WA,-7.99
7,2025-04-04 00:00:00,12345678912345678912351,MITSUWA MRKTPLACE IR IRVINE CA,IRVINE CA,-22.08
8,2025-03-27 00:00:00,12345678912345678912352,Microsoft*Xbox Live Gold 425-6816830 WA,425-6816830 WA,-9.99
